R magic
=============

In [ ]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Generate dimension tables
================================

In [133]:
%%R
cheese_table <- data.frame(key=1:3,
            name=c("Swiss", "Cheddar", "Mozzzarella"),
             supplier_key=1:3)
cheese_table

  key        name supplier_key
1   1       Swiss            1
2   2     Cheddar            2
3   3 Mozzzarella            3


In [134]:
%%R
dough_table <- data.frame(key=1:3,
            name=c("whole wheat thin", "white regular", "stuffed crust"),
             supplier_key=1:3)
dough_table

  key             name supplier_key
1   1 whole wheat thin            1
2   2    white regular            2
3   3    stuffed crust            3


In [135]:
%%R
size_table <- data.frame(key=1:4,
             name=c("small", "medium", "large", "xlarge"),
             specification=c("8 inches", "12 inches", "14 inches", "16 inches"))
size_table

  key   name specification
1   1  small      8 inches
2   2 medium     12 inches
3   3  large     14 inches
4   4 xlarge     16 inches


In [136]:
%%R
supplier_table <- data.frame(key=1:3,
             name=c("Ubisoft", "EA", "KONAMI"))
supplier_table

  key    name
1   1 Ubisoft
2   2      EA
3   3  KONAMI


In [138]:
%%R
city_table <- data.frame(key=1:4,
             name=c("Ottawa", "Toronto", "Edmonton", "Calgary"),
             province_key=1:4)
city_table

  key     name province_key
1   1   Ottawa            1
2   2  Toronto            2
3   3 Edmonton            3
4   4  Calgary            4


In [139]:
%%R
location_table <- data.frame(key=1:4,
             name=c("1123, Holland Street", "304, Baseline Street", "5463, Friday Street", "43, Somerset Street"),
             city_key=1:4)
location_table

  key                 name city_key
1   1 1123, Holland Street        1
2   2 304, Baseline Street        2
3   3  5463, Friday Street        3
4   4  43, Somerset Street        4


In [141]:
%%R
province_table <- data.frame(key=1:2,
             name=c("Ontario", "Alberta"),
             country=c("Canada", "Canada"))
province_table

  key    name country
1   1 Ontario  Canada
2   2 Alberta  Canada


In [142]:
%%R
store_table <- data.frame(key=1:4,
             location_key=1:4,
             manager=c("Mike", "Bob", "Mary", "Jessie"))
store_table

  key location_key manager
1   1            1    Mike
2   2            2     Bob
3   3            3    Mary
4   4            4  Jessie


In [143]:
%%R
topping_table <- data.frame(key=1:4,
             name=c("tomatoes", "pepper", "onions", "pepperoni"))
topping_table

  key      name
1   1  tomatoes
2   2    pepper
3   3    onions
4   4 pepperoni


In [144]:
%%R
month_table <- 
  data.frame(key=1:12,
            desc=c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"),
            quarter=c("Q1","Q1","Q1","Q2","Q2","Q2","Q3","Q3","Q3","Q4","Q4","Q4"))
month_table

   key desc quarter
1    1  Jan      Q1
2    2  Feb      Q1
3    3  Mar      Q1
4    4  Apr      Q2
5    5  May      Q2
6    6  Jun      Q2
7    7  Jul      Q3
8    8  Aug      Q3
9    9  Sep      Q3
10  10  Oct      Q4
11  11  Nov      Q4
12  12  Dec      Q4


generate fact table
============================

sample from all necessary dimension tables

In [148]:
%%R
sample_num = 100
store <- sample(store_table$key, sample_num, replace=T)
month <- sample(month_table$key, sample_num, replace=T)
cheese <- sample(cheese_table$key, sample_num, replace=T)
topping <- sample(topping_table$key, sample_num, replace=T)
size <- sample(size_table$key, sample_num, replace=T)
dough <- sample(dough_table$key, sample_num, replace=T)

  [1] 2 2 1 3 3 2 1 2 1 3 2 3 2 2 2 1 2 3 2 2 3 2 3 1 3 3 2 3 1 1 1 3 3 3 1 1 3
 [38] 1 2 2 1 3 3 3 1 2 3 3 1 2 1 2 3 1 2 3 1 3 3 3 1 2 3 2 3 1 3 2 3 2 1 1 3 3
 [75] 3 3 1 3 2 1 1 1 2 3 3 1 2 3 2 1 1 3 2 2 1 1 3 3 3 3


the profit is determined by size, so we have

In [150]:
%%R
profit <- 1 * size

and we hope the amount of xlarge pizza will increase along with the increase of the month
$$ amount = month * size\_weights(size) + noise$$

In [156]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 1
   } else if(x == 3){
       w <- 2
   }
   else{
       w <- 5
   }
   w
}

weight <- sapply(size, size_weights)
print(head(size, 20))
print(head(weight, 20))
noise <- sample(1:10, sample_num, replace=T)
amount <- month*weight+noise
print(head(amount, 20))

 [1] 4 2 1 3 2 4 1 2 1 3 3 1 1 1 4 4 4 4 1 4
 [1] 5 1 1 2 1 5 1 1 1 2 2 1 1 1 5 5 5 5 1 5
 [1] 63 11 14 17 15 49 11 10  6 14 19 11 14 10 42 25 23 16  5 13


we assemble them together to get a function

In [163]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 1
   } else if(x == 3){
       w <- 2
   }
   else{
       w <- 5
   }
   w
}

generate_orders <- function(sample_num){
    store <- sample(store_table$key, sample_num, replace=T)
    month <- sample(month_table$key, sample_num, replace=T)
    cheese <- sample(cheese_table$key, sample_num, replace=T)
    topping <- sample(topping_table$key, sample_num, replace=T)
    size <- sample(size_table$key, sample_num, replace=T)
    dough <- sample(dough_table$key, sample_num, replace=T)

    profit <- 1 * size

    weight <- sapply(size, size_weights)
    noise <- sample(1:10, sample_num, replace=T)
    amount <- month*weight+noise

    # generate the fact table(order)
    order_table <- data.frame(store_key=store,
                      month_key=month,
                      size_key=size,
                      dough_key=dough,
                      cheese_key=cheese,
                      topping_key=topping,
                      amount=amount,
                      profit=profit)
    order_table <- order_table[order(order_table$store_key, -order_table$month_key),]
    row.names(order_table) <- NULL
    return(order_table)
}

new let's generate a fact table

In [166]:
%%R
order_table <- generate_orders(5000)
head(order_table,20)

   store_key month_key size_key dough_key cheese_key topping_key amount profit
1          1        12        2         2          1           2     19      2
2          1        12        1         2          3           2     18      1
3          1        12        3         1          1           2     27      3
4          1        12        3         3          1           3     25      3
5          1        12        1         3          1           3     15      1
6          1        12        3         3          3           2     28      3
7          1        12        4         3          1           4     63      4
8          1        12        1         1          1           3     13      1
9          1        12        4         3          3           1     70      4
10         1        12        2         2          3           1     13      2
11         1        12        3         3          1           1     25      3
12         1        12        1         3          2

OLAP

In [167]:
%%R
dimension <- c("store_key", "month_key", "size_key", "dough_key", "cheese_key", "topping_key")
revenue_cube <- 
    tapply(order_table$amount, 
           order_table[,dimension], 
           FUN=function(x){return(sum(x))})


In [169]:
%%R
revenue_cube

, , size_key = 1, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1  6 16 NA NA NA NA NA 12 12 29 NA 16
        2 NA NA 12  7 NA 13 25 NA NA NA NA 19
        3  5 NA NA NA NA 16 13 NA NA NA 15 NA
        4 NA  6 13  5 NA NA 12 23 NA NA NA NA

, , size_key = 2, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 16  4 NA NA 19 11  8 NA NA 12 12 21
        2  5 NA  5 NA 43 29 11 NA NA 15 14 NA
        3 17  9 NA NA 14 23 NA 27 NA 15 20 14
        4 NA 10  4 11  7 NA 13 NA NA NA NA NA

, , size_key = 3, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 NA NA NA NA 11 22 19 39 NA 23 NA NA
        2 18 19  9 12 NA 35 24 21 26 NA NA 30
        3 19 21 NA NA 61 21 NA NA NA NA 27 NA
        4  8 NA 10 36 18 19 17 NA 53 51 NA NA

, , size_key = 4, dough_key = 1, cheese_key = 1, top